In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Informations sur ladataset

#### Imaginez qu'un fournisseur de télécommunications a segmenté sa clientèle en fonction des modèles d'utilisation des services, en classant les clients en quatre groupes. Si les données démographiques peuvent être utilisées pour prédire l'appartenance à un groupe, l'entreprise peut personnaliser les offres pour des clients potentiels individuels. C'est un problème de classification. Autrement dit, étant donné l'ensemble de données, avec des étiquettes prédéfinies, nous devons construire un modèle à utiliser pour prédire la classe d'un cas nouveau ou inconnu.

#### L'exemple se concentre sur l'utilisation de données démographiques, telles que la région, l'âge et le mariage, pour prédire les modèles d'utilisation.

#### Le champ cible, appelé custcat, a quatre valeurs possibles qui correspondent aux quatre groupes de clients, comme suit: 1- Service de base 2- E-Service 3- Plus Service 4- Service total

#### Notre objectif est de construire un classifieur, de prédire la classe des cas inconnus. Nous utiliserons un type de classification spécifique appelé K plus proche voisin.

In [ ]:
df = pd.read_csv("../input/telecustsclasses/teleCust1000t.csv")

In [ ]:
df

In [ ]:
df['custcat'].value_counts()

In [ ]:
df.hist(column='income', bins=50)

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
X = df[['region', 'tenure','age', 'marital', 'address', 'income', 'ed', 'employ','retire', 'gender', 'reside']] .values 


In [ ]:
X[0:5]

In [ ]:
y = df['custcat'].values
y[0:5]

## Normalisation 

In [ ]:
X = preprocessing.StandardScaler().fit(X).transform(X.astype(float))
X[0:5]

## Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

## Classification

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
k = 4
#Training
neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
neigh


In [ ]:
yhat = neigh.predict(X_test)
yhat[0:5]

### Accuracy

#### avec K=4

In [ ]:
from sklearn import metrics
print("Train set Accuracy: ", metrics.accuracy_score(y_train, neigh.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, yhat))

#### avec K=6

In [ ]:
k = 6
neigh6 = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
yhat6 = neigh6.predict(X_test)
print("Train set Accuracy: ", metrics.accuracy_score(y_train, neigh6.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, yhat6))

In [ ]:
Ks = 10
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
ConfustionMx = [];
for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    yhat=neigh.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)

    
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])

mean_acc

In [ ]:
plt.plot(range(1,Ks),mean_acc,'g')
plt.fill_between(range(1,Ks),mean_acc - 1 * std_acc,mean_acc + 1 * std_acc, alpha=0.10)
plt.legend(('Accuracy ', '+/- 3xstd'))
plt.ylabel('Accuracy ')
plt.xlabel('Number of Nabors (K)')
plt.tight_layout()
plt.show()

In [ ]:
print( "The best accuracy was with", mean_acc.max(), "with k=", mean_acc.argmax()+1) 